# --- Day 17 Pyroclastic Flow ---

https://adventofcode.com/2022/day/17

## Get Input Data

In [11]:
def parse_input(filename):
    """Parse input data for puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    
    """
    with open(f'../inputs/{filename}.txt') as _file:
        jet_pattern = [int(c.replace("<", "-1").replace(">", "1")) for c in _file.read()]

    return jet_pattern

Replace `">"`` with `1`` and `"<"` with `-1`, as these will represent moves *right* and moves *left*, respectively.

In [12]:
parse_input("test_jet_pattern")

[1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1]

### Define `rocks` data structure

In [13]:
import numpy as np

In [ ]:
rocks = {
    0 : {
        "max_left" : 0,
        "max_right" : 0,
        "struct" : None
    },
    1: {
        "max_left" : 0,
        "max_right" : 0,
        "struct" : None
    },
    2: {
        "max_left" : 0,
        "max_right" : 0,
        "struct" : None
    },
    3: {
        "max_left" : 0,
        "max_right" : 0,
        "struct" : None
    },
    4: {
        "max_left" : 0,
        "max_right" : 0,
        "struct" : None
    },
}

## Part 1
---

### Run on Test Data

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data